# Resource Assignment Problem (RAP) <br> Parse a toy dataset and solve the problem

## Load dependencies

In [1]:
%%time
from timeit import default_timer as timer
from time import process_time#, process_time_ns
from os.path import exists
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
from itertools import product
import re
# import solvers
import gurobipy as gp  # conda install -c gurobi gurobi
from gurobipy import GRB
import pulp as pl  # pip install pulp
import glpk  # conda install -c conda-forge glpk

CPU times: total: 406 ms
Wall time: 436 ms


## Set input parameters

In [2]:
print('Setting input parameters...')
ttime = timer()

config = {
          'data_filename': '20220527__[Optim achats] Dataset toy.xlsx',  # 100 demands, 4 PN categories, 40 PN ids, 3 suppliers
          'problem_export': True,  # export problem to LP file and MPS file
          'gurobi_export': True,  # export Gurobi solution
          'greedy_export': True,  # export Greedy solution
         }

print('Elapsed time is %.4f seconds.' % (timer()-ttime))

Setting input parameters...
Elapsed time is 0.0001 seconds.


## Load data

In [3]:
print('Loading data...')
ttime = timer()

# Dataframes from XL file
df_dem = pd.read_excel(config['data_filename'], index_col=0, sheet_name='Demand')
df_cat = pd.read_excel(config['data_filename'], index_col=0, sheet_name='Catalog')
df_ms  = pd.read_excel(config['data_filename'], index_col=0, sheet_name='Market share')
df_vd  = pd.read_excel(config['data_filename'], index_col=0, sheet_name='Volume discount')

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Sample & modify input data
# df_dem = df_dem.head(50)
# df_vd = df_vd.head(3)
# df_cat.loc[df_cat.loc[:,'UnitPrice']==9999,'UnitPrice'] = 10
# df_cat.loc[df_cat.loc[:,'Supplier']==57,'UnitPrice'] = 1000
# df_vd.loc[0,'Threshold'] = 1e3
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------

print('<- ''df_dem''.shape = %s' % str(df_dem.shape))
print('<- ''df_cat''.shape = %s' % str(df_cat.shape))
print('<- ''df_ms''.shape = %s'  % str(df_ms.shape))
print('<- ''df_vd''.shape = %s'  % str(df_vd.shape))

print('Elapsed time is %.4f seconds.' % (timer()-ttime))

Loading data...
<- df_dem.shape = (100, 4)
<- df_cat.shape = (120, 3)
<- df_ms.shape = (12, 6)
<- df_vd.shape = (12, 5)
Elapsed time is 0.4342 seconds.


## Preprocess dataframes

In [4]:
print('Preprocessing data...')
ttime = timer()

# Get unique values 
dm = df_dem['DemandID'].drop_duplicates().sort_values()
su = pd.concat([df_cat['Supplier'], df_ms['Supplier']]).drop_duplicates().sort_values()
ms = df_ms['MS_ID'].drop_duplicates().sort_values()
vd = df_vd['VD_ID'].drop_duplicates().sort_values()

# Allocation dataframe
df_alloc = df_cat.pivot(index='PN', columns='Supplier', values='UnitPrice').add_prefix('Supplier_').replace(9999, None)
df_alloc = df_dem.merge(df_alloc, on='PN', how='left')

# Exclude demands with no prices + print them
df_alloc_no = df_alloc[df_alloc.loc[:,df_alloc.columns.str.startswith('Supplier_')].sum(axis=1)==0]
if df_alloc_no.empty:
    print('-> no demands with no prices')
else:
    print('-> %d demands with no prices: %s' % (df_alloc_no.shape[0], str(df_alloc_no['DemandID'].values)))
    df_alloc = df_alloc[~df_alloc['DemandID'].isin(df_alloc_no['DemandID'])].reset_index()

# Define features
price = {pair: df_alloc['Supplier_' + str(pair[1])][df_alloc['DemandID']==pair[0]].values[0] for pair in product(dm, su)}
price = {key: val for key, val in price.items() if val!=9999 and val is not None}
quantity  = dict(zip(df_alloc['DemandID'], df_alloc['Quantity']))
ms_min    = dict(zip(df_ms['MS_ID'], df_ms['MS_Min']))
ms_max    = dict(zip(df_ms['MS_ID'], df_ms['MS_Max']))
penalty   = dict(zip(df_ms['MS_ID'], df_ms['Penalty']))
threshold = dict(zip(df_vd['VD_ID'], df_vd['Threshold']))
reduction = dict(zip(df_vd['VD_ID'], df_vd['Pct reduction']))

# dict of '(demand, supplier): reduction' with reduced price if any
price_red = {}
for v in vd:
    pn_su_red = df_vd.query('VD_ID==@v')[['PN category', 'Supplier', 'Pct reduction']].values[0]
    price_red.update({pair: pn_su_red[2] for pair in [(d, pn_su_red[1]) for d in df_dem.query('`PN category`==@pn_su_red[0]')['DemandID']]})
price_red = {key: val*(1-price_red[key]/1e2) for key, val in price.items() if key in price_red.keys()}

print('Elapsed time is %.4f seconds.' % (timer()-ttime))

Preprocessing data...
-> no demands with no prices
Elapsed time is 0.4734 seconds.


## Set up the problem

The LP formulation (including constraints for volume discount and market share) is:

$ 
\begin{array}{ccl}
\mbox{Minimize}_{(x,y,z)} & & \sum_{d,s} x_{ds} \cdot qty(d) \cdot price(d,s) + \sum_{d,s} y_{ds} \cdot qty(d) \cdot price\_reduc(d,s) + \sum_m z_m \cdot penalty(m) \\
\mbox{subject to} & \forall d, & \sum_s x_{ds} + \sum_s y_{ds} = 1 \\ 
& \forall v, & 0 \leq \sum_{d | PNcat(d)} x_{d,s(v)} \cdot qty(d) \leq (\tau(v)-1) \cdot (1 - w_v) \\
& & \tau(v) \cdot w_v \leq \sum_{d | PNcat(d)} y_{d,s(v)} \cdot qty(d) \leq \left( \sum_{d} qty(d) \right) \cdot w_v \\
& & w_v \in \{0,1\} \\
& \forall m & MS\_min(m) \leq \dfrac{\sum_{d | PNcat(d)} x_{d,s(m)} \cdot qty(d) + z_m }{\sum_{d | PNcat(d)} qty(d)} \leq MS\_max(m) \\
& & z_m \geq 0 \\
& \forall d,s & x_{ds} \in \{0,1\} \\
& & y_{ds} \in \{0,1\} \\
\end{array}
$

In [5]:
print('Setting up the problem...')
ttime = timer()

mdl = gp.Model('Toy problem')
# VARIABLES -------------------------------------------------------------------
x = mdl.addVars(price.keys(), vtype=GRB.BINARY, name='x')
y = mdl.addVars(price_red.keys(), vtype=GRB.BINARY, name='y')
w = mdl.addVars(vd, vtype=GRB.BINARY, name='w')
z = mdl.addVars(ms, name='z')
# CONSTRAINTS -----------------------------------------------------------------
mdl.addConstrs((x.sum(d, '*') + y.sum(d, '*') == 1 for d in dm), name='cstr_demand')
bigM = sum(quantity.values())
for v in vd:  # Volume discount constraints
    pn_su = df_vd.query('VD_ID==@v')[['PN category', 'Supplier']].values[0]
    dm_qt = df_alloc.query('`PN category`==@pn_su[0]')[['DemandID', 'Quantity']]
    a_qty = gp.quicksum(quantity[d]*x[d, s] for d, s in price.keys() if d in dm_qt['DemandID'].values and s==pn_su[1])
    mdl.addConstr(a_qty <= (threshold[v]-1)*(1-w[v]), name='cstr_vd_ko[%d]' % v)
    a_qty = gp.quicksum(quantity[d]*y[d, s] for d, s in price_red.keys() if d in dm_qt['DemandID'].values and s==pn_su[1])
    mdl.addConstr(a_qty >= threshold[v]*w[v], name='cstr_vd_ok_min[%d]' % v)
    mdl.addConstr(a_qty <= bigM*w[v], name='cstr_vd_ok_max[%d]' % v)
for m in ms:  # Market share constraints
    pn_su = df_ms.query('MS_ID==@m')[['PN category', 'Supplier']].values[0]
    dm_qt = df_alloc.query('`PN category`==@pn_su[0]')[['DemandID', 'Quantity']]
    a_qty = gp.quicksum(quantity[d]*(x[d, s]+y[d, s]) for d, s in price.keys() if d in dm_qt['DemandID'].values and s==pn_su[1])
    mdl.addConstr((a_qty + z[m]) / np.sum(dm_qt['Quantity']) >= ms_min[m] / 1e2, name='cstr_ms_min[%d]' % m)
    mdl.addConstr((a_qty + z[m]) / np.sum(dm_qt['Quantity']) <= ms_max[m] / 1e2, name='cstr_ms_max[%d]' % m)
# OBJECTIVE FUNCTION ----------------------------------------------------------
mdl.setObjective(gp.quicksum(quantity[d]*price[d, s]*x[d, s] for d, s in price.keys())
                 + gp.quicksum(quantity[d]*price_red[d, s]*y[d, s] for d, s in price_red.keys())
                 + z.prod(penalty)
                 , GRB.MINIMIZE)

# save model for inspection
if 'problem_export' in config.keys() and config['problem_export']:
    for ext in ['.lp', '.mps']:
        tmp_fname = config['data_filename'].replace('.xlsx', '__problem' + ext)
        mdl.write(tmp_fname)
        print('-> %s' % tmp_fname)
print('Elapsed time is %.4f seconds.' % (timer()-ttime))

Setting up the problem...
Restricted license - for non-production use only - expires 2023-10-25
-> 20220527__[Optim achats] Dataset toy__problem.lp
-> 20220527__[Optim achats] Dataset toy__problem.mps
Elapsed time is 0.4037 seconds.


In [6]:
def get_results(assign_x, assign_y, solver_name, verbose=True):
    # df_alloc_res -------------------------------------------------------------------
    df_assign = pd.DataFrame(list(assign_x.keys()) + list(assign_y.keys()), columns=['DemandID', 'SuggestedSupplier'])
    df_alloc_res = df_alloc.merge(df_assign, on='DemandID', how='left')
    df_alloc_res['GotDiscount'] = df_alloc_res['DemandID'].apply(lambda x: x in [d for d, _ in assign_y.keys()]).astype(int).replace(0, '')
    df_alloc_res['SuggestedPrice'] = list(zip(df_alloc_res['DemandID'], df_alloc_res['SuggestedSupplier']))
    for i, b in enumerate(df_alloc_res['GotDiscount']):
        if not b:
            df_alloc_res.loc[i, 'SuggestedPrice'] = price[df_alloc_res.loc[i, 'SuggestedPrice']]
        else:
            df_alloc_res.loc[i, 'SuggestedPrice'] = price_red[df_alloc_res.loc[i, 'SuggestedPrice']]
    df_alloc_res['TotalCost'] = df_alloc_res['Quantity'] * df_alloc_res['SuggestedPrice']
    df_alloc_res['CheapestPrice'] = df_alloc_res['PN'].apply(lambda x: min(df_cat.query('PN==@x')['UnitPrice']))
    df_alloc_res = df_alloc_res.merge(df_cat, left_on=['PN', 'CheapestPrice'], right_on=['PN', 'UnitPrice'], how='left'
                                     ).drop_duplicates(['DemandID'], keep='first').drop('UnitPrice', axis=1)
    df_alloc_res.rename(columns={'Supplier': 'CheapestSupplier'}, inplace=True)
    # df_mshare_res ------------------------------------------------------------------
    df_ms_res = df_ms.copy().reindex(columns=df_ms.columns.tolist() + ['SuggestedMarketShare', 'ClaimQTY'])
    for m in ms:
        pn_su = df_ms.query('MS_ID==@m')[['PN category', 'Supplier']].values[0]
        sum_aqty = df_alloc_res.query('`PN category`==@pn_su[0] & SuggestedSupplier==@pn_su[1]')['Quantity'].sum()
        sum_qty = df_alloc_res.query('`PN category`==@pn_su[0]')['Quantity'].sum()
        val_ms = sum_aqty / sum_qty * 1e2
        df_ms_res.loc[df_ms_res['MS_ID']==m, 'SuggestedMarketShare'] = val_ms
        min_max = df_ms_res.loc[df_ms_res['MS_ID']==m, ['MS_Min', 'MS_Max']].values[0]
        if val_ms < min_max[0]:
            df_ms_res.loc[df_ms_res['MS_ID']==m, 'ClaimQTY'] = sum_qty * (min_max[0] - val_ms) / 1e2
        elif val_ms > min_max[1]:
            df_ms_res.loc[df_ms_res['MS_ID']==m, 'ClaimQTY'] = sum_qty * (val_ms - min_max[1]) / 1e2
        else:
            df_ms_res.loc[df_ms_res['MS_ID']==m, 'ClaimQTY'] = 0
    df_ms_res['SuggestedMarketShare'] = round(df_ms_res['SuggestedMarketShare'], 2)
    df_ms_res['ClaimCost'] = df_ms_res['ClaimQTY'] * df_ms_res['Penalty']
    # kpi ------------------------------------------------------------------
    kpi = {}
    kpi['solver_name'] = solver_name
    kpi['nb_assign'] = len(assign_x) + len(assign_y)
    kpi['nb_assign_tot'] = len(df_dem)
    kpi['nb_ms_respect'] = np.sum([df_ms_res['ClaimQTY']>1e-6])  # nb of respected ms contracts
    kpi['nb_ms_tot'] = len(df_ms)
    kpi['total_price'] = df_alloc_res['TotalCost'].sum()
    kpi['total_cost'] = df_alloc_res['TotalCost'].sum() + df_ms_res['ClaimCost'].sum()
    # verbose --------------------------------------------------------------
    if verbose:
        print('   Number of dem assignments \t= %d / %d (%d without and %d with reduction)' % (kpi['nb_assign'], df_dem.shape[0], len(assign_x), len(assign_y)))
        print('   Number of market share ok \t= %d / %d' % (kpi['nb_ms_respect'], len(ms)))
        print('   Total price (assignement) \t= %.6f' % kpi['total_price'])
        print('   Optimal objective function \t= %.6f' % kpi['total_cost'])
    return df_alloc_res, df_ms_res, kpi

def get_cputime(s_optimize, nb_run=1e3):
    nb_run = int(nb_run)
    cputime = process_time()  # in sec
    eval('[%s for i in range(%d)]' % (s_optimize, nb_run))
    cputime = (process_time() - cputime) / nb_run * 1e6  # average cputime in microseconds
    return nb_run, cputime

## Solve with Gurobi

In [7]:
print('Solving with Gurobi...\n')
ttime = timer()

# run optimization engine
mdl.Params.LogToConsole = 1
mdl.optimize()
mdl.Params.LogToConsole = 0

print()
if mdl.Status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
elif mdl.Status == GRB.OPTIMAL:
    gurobi_x = {key: price[key] for key, value in mdl.getAttr('X', x).items() if value>1e-6}
    gurobi_y = {key: price_red[key] for key, value in mdl.getAttr('X', y).items() if value>1e-6}
    gurobi_assign = [key for key, value in mdl.getAttr('X', x).items() if value>1e-6]
    gurobi_alloc, gurobi_ms, gurobi_kpi = get_results(gurobi_x, gurobi_y, 'Gurobi ' + str(gp.gurobi.version()))
    gurobi_kpi['cputime_number'], gurobi_kpi['cputime_microsec'] = get_cputime('mdl.optimize()')
    print('   Optimization duration \t= %.2f microsec (average over %d runs)' % (gurobi_kpi['cputime_microsec'], gurobi_kpi['cputime_number']))
else:  #if mdl.Status != GRB.INF_OR_UNBD and mdl.Status != GRB.INFEASIBLE:
    print('Optimization was stopped with status %d' % mdl.Status)

# Export solution
if 'gurobi_export' in config.keys() and config['gurobi_export']:
    tmp_fname = config['data_filename'].replace('.xlsx', '__solution_gurobi.xlsx')
    with pd.ExcelWriter(tmp_fname) as fid:
        gurobi_alloc.to_excel(fid, sheet_name='Allocation')
        gurobi_ms.to_excel(fid, sheet_name='Market share')
    print('\n-> ' + tmp_fname)
    # tmp_fname = config['data_filename'].replace('.xlsx', '__solution_gurobi.sol')
    # mdl.write(tmp_fname)
    # print('-> ' + tmp_fname)

print('\nElapsed time is %.4f seconds.' % (timer()-ttime))

Solving with Gurobi...

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 160 rows, 566 columns and 2499 nonzeros
Model fingerprint: 0x81e1b217
Variable types: 12 continuous, 554 integer (554 binary)
Coefficient statistics:
  Matrix range     [3e-04, 1e+04]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-02, 1e+03]
Found heuristic solution: objective 61401.760000
Presolve removed 117 rows and 391 columns
Presolve time: 0.03s
Presolved: 43 rows, 175 columns, 666 nonzeros
Found heuristic solution: objective 35499.800000
Variable types: 0 continuous, 175 integer (173 binary)
Found heuristic solution: objective 33118.800000

Root relaxation: objective 2.923730e+04, 36 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | 

## Solve with heuristic (greedy)

In [8]:
def greedy_solve(w_price, w_price_red):  # select iteratively the cheapest demand
    assign_x = list()
    assign_y = list()
    while len(w_price) > 0:
        selection1 = min(w_price, key=w_price.get)
        selection2 = min(w_price_red, key=w_price_red.get)
        if selection1 < selection2:
            assign_x.append(selection1)
            selection = selection1
        else:
            assign_y.append(selection2)
            selection = selection2
        w_price = {key: val for key, val in w_price.items() if key[0]!=selection[0]}
        w_price_red = {key: val for key, val in w_price_red.items() if key[0]!=selection[0]}
    assign_x = {key: price[key] for key in assign_x}
    assign_y = {key: price_red[key] for key in assign_y}
    return assign_x, assign_y

In [9]:
print('Solving with heuristic (greedy)...\n')
ttime = timer()

# Greedy solution
w_price = {(d, s): price[d, s] * quantity[d] for d, s in price.keys()}
w_price_red = {(d, s): price_red[d, s] * quantity[d] for d, s in price_red.keys()}
greedy_x, greedy_y = greedy_solve(w_price, w_price_red)
greedy_alloc, greedy_ms, greedy_kpi = get_results(greedy_x, greedy_y, 'Greedy')
greedy_kpi['cputime_number'], greedy_kpi['cputime_microsec'] = get_cputime('greedy_solve(w_price, w_price_red)')
print('   Optimization duration \t= %.2f microsec (average over %d runs)' % (greedy_kpi['cputime_microsec'], greedy_kpi['cputime_number']))

# Export solution
if 'greedy_export' in config.keys() and config['greedy_export']:
    tmp_fname = config['data_filename'].replace('.xlsx', '__solution_greedy.xlsx')
    with pd.ExcelWriter(tmp_fname) as fid:  
        greedy_alloc.to_excel(fid, sheet_name='Allocation')
        greedy_ms.to_excel(fid, sheet_name='Market share')
    print('\n-> ' + tmp_fname)

print('\nElapsed time is %.4f seconds.' % (timer()-ttime))

Solving with heuristic (greedy)...

   Number of dem assignments 	= 100 / 100 (39 without and 61 with reduction)
   Number of market share ok 	= 6 / 12
   Total price (assignement) 	= 25381.000000
   Optimal objective function 	= 36157.660000
   Optimization duration 	= 4140.62 microsec (average over 1000 runs)

-> 20220527__[Optim achats] Dataset toy__solution_greedy.xlsx

Elapsed time is 4.9282 seconds.


## Solve with Pulp

In [10]:
# get assignement values from Pulp variables
def get_assign_pulp(variables):
    assign_x = [tuple(map(int, re.sub('x|\[|\]|_', '', v.name).split(','))) for v in variables if v.name.startswith('x') and v.varValue==1]
    assign_y = [tuple(map(int, re.sub('y|\[|\]|_', '', v.name).split(','))) for v in variables if v.name.startswith('y') and v.varValue==1]
    assign_x = {key: price[key] for key in assign_x}
    assign_y = {key: price_red[key] for key in assign_y}
    return assign_x, assign_y

In [11]:
print('Solving with Pulp...\n')
ttime = timer()

# write MPS problem if not exists
tmp_fname = config['data_filename'].replace('.xlsx', '__problem.mps')
if not exists(tmp_fname):
    mdl.write(tmp_fname)
    print('-> %s' % tmp_fname)

# read MPS problem
_, pl_mdl = pl.LpProblem.fromMPS(tmp_fname)
    
pulp_kpi = {}
pulp_nb_run = 1e1
for solver in pl.listSolvers(onlyAvailable=True):
    print(' * Solver \'%s\'' % solver)
    s_solve_cmd = 'pl_mdl.solve(pl.%s(msg=0))' % solver
    eval(s_solve_cmd)
    pulp_x, pulp_y = get_assign_pulp(pl_mdl.variables())
    pulp_alloc, pulp_ms, pulp_kpi[solver] = get_results(pulp_x, pulp_y, solver)
    pulp_kpi[solver]['cputime_number'], pulp_kpi[solver]['cputime_microsec'] = get_cputime(s_solve_cmd, pulp_nb_run)
    print('   Optimization duration \t= %.2f microsec (average over %d runs)' % (pulp_kpi[solver]['cputime_microsec'], pulp_kpi[solver]['cputime_number']))
    
print('\nElapsed time is %.4f seconds.' % (timer()-ttime))

Solving with Pulp...

No parameters matching '_test' found
 * Solver 'GLPK_CMD'
   Number of dem assignments 	= 100 / 100 (29 without and 71 with reduction)
   Number of market share ok 	= 7 / 12
   Total price (assignement) 	= 29167.700000
   Optimal objective function 	= 29569.800000
   Optimization duration 	= 17187.50 microsec (average over 10 runs)
 * Solver 'GUROBI'
   Number of dem assignments 	= 100 / 100 (29 without and 71 with reduction)
   Number of market share ok 	= 7 / 12
   Total price (assignement) 	= 28879.700000
   Optimal objective function 	= 29569.800000
   Optimization duration 	= 131250.00 microsec (average over 10 runs)
 * Solver 'GUROBI_CMD'
   Number of dem assignments 	= 100 / 100 (29 without and 71 with reduction)
   Number of market share ok 	= 7 / 12
   Total price (assignement) 	= 29167.700000
   Optimal objective function 	= 29569.800000
   Optimization duration 	= 15625.00 microsec (average over 10 runs)
 * Solver 'PULP_CBC_CMD'
   Number of dem assignm

## Solve with GLPK

In [12]:
# get assignement values from GLPK variables
def get_assign_glpk(cols):
    assign_x = [tuple(map(int, re.sub('x|\[|\]|_', '', c.name).split(','))) for c in cols if c.name.startswith('x') and c.primal==1]
    assign_y = [tuple(map(int, re.sub('y|\[|\]|_', '', c.name).split(','))) for c in cols if c.name.startswith('y') and c.primal==1]
    assign_x = {key: price[key] for key in assign_x}
    assign_y = {key: price_red[key] for key in assign_y}
    return assign_x, assign_y

In [13]:
print('Solving with GLPK...\n')  # GNU Linear Programming Kit
ttime = timer()

# write MPS problem if not exists
tmp_fname = config['data_filename'].replace('.xlsx', '__problem.mps')
if not exists(tmp_fname):
    mdl.write(tmp_fname)
    print('-> %s' % tmp_fname)

# read MPS problem
lp = glpk.LPX(freemps=tmp_fname) # Read free MPS format file
# lp = glpk.LPX(glp=tmp_fname) # Read GNU LP format file

# lp.interior() # Solve problem with the interior-point method

lp.simplex()  # Solve this LP with the s/implex method
# lp.exact()  # Solve problem with an exact arithmetic using simplex method
lp.integer()  # Solve MIP problem with the B&B method
# lp.intopt() # Solve MIP problem with the advanced B&B solver

glpk_nb_run = 1e1
glpk_x, glpk_y = get_assign_glpk(lp.cols)
glpk_alloc, glpk_ms, glpk_kpi = get_results(glpk_x, glpk_y, 'GLPK ' + str(glpk.env.version))
glpk_kpi['cputime_number'], glpk_kpi['cputime_microsec'] = get_cputime('(lp.simplex(), lp.integer())', glpk_nb_run)

print('\nElapsed time is %.4f seconds.' % (timer()-ttime))

Solving with GLPK...

   Number of dem assignments 	= 100 / 100 (29 without and 71 with reduction)
   Number of market share ok 	= 7 / 12
   Total price (assignement) 	= 28879.700000
   Optimal objective function 	= 29569.800000

Elapsed time is 22.9253 seconds.


## Other solvers

* SCIP https://www.scipopt.org/#download 
* Scikit-decide https://github.com/airbus/scikit-decide
* Pyomo https://github.com/Pyomo/pyomo/

## Compare solvers

In [14]:
df_compare = []
df_compare.append(pd.DataFrame([gurobi_kpi]))
df_compare.append(pd.DataFrame([greedy_kpi]))
if 'pulp_kpi' in locals():
    for solver in pulp_kpi.keys():
        df_compare.append(pd.DataFrame([pulp_kpi[solver]]))
if 'glpk_kpi' in locals():
    df_compare.append(pd.DataFrame([glpk_kpi]))
df_compare = pd.concat(df_compare).set_index('solver_name')

df_compare

,nb_assign,nb_assign_tot,nb_ms_respect,nb_ms_tot,total_price,total_cost,cputime_number,cputime_microsec
solver_name,,,,,,,,
"Gurobi (9, 5, 1)",100,100,7,12,28879.7,29569.80,1000,62.500
Greedy,100,100,6,12,25381.0,36157.66,1000,4140.625
GLPK_CMD,100,100,7,12,29167.7,29569.80,10,17187.500
GUROBI,100,100,7,12,28879.7,29569.80,10,131250.000
GUROBI_CMD,100,100,7,12,29167.7,29569.80,10,15625.000
PULP_CBC_CMD,100,100,7,12,28879.7,29569.80,10,12500.000
"GLPK (4, 65)",100,100,7,12,28879.7,29569.80,10,2010937.500
